## Краулер, который собирает тексты с новостного ресурса.
Выбираете себе одну газету и скачиваете оттуда статьи для корпуса. Вид статьи описан в доке. Давайте статей будет до 1000. (http://www.belrab.ru/)


!!!Не стала выводить автора, т.к в разных статьях он в разных местах по тексту, хотя можно было бы использовать регистр, т.к. фамилия автора всегда написана большими буквами. Но точно так же маркируется фамилия фотографа и т.п. и в той же части, где автор

In [97]:
import requests
import re
from bs4 import BeautifulSoup
from IPython.display import HTML, display

In [98]:
def article_links(archieve_url):
    n = 0

    article_links = []

    archieve_page_url = 'http://www.belrab.ru' + archieve_url + '?start=' + str(n)
    req = requests.get(archieve_page_url)
    soup = BeautifulSoup(req.text, 'lxml')
    end_page = 'http://www.belrab.ru' + soup.find('a', attrs = {'title':'В конец'})['href']
    
    
    while archieve_page_url != end_page:
        
        archieve_page_url = 'http://www.belrab.ru' + archieve_url + '?start=' + str(n)
        req = requests.get(archieve_page_url)
        soup = BeautifulSoup(req.text, 'lxml')
        
        for h in soup.findAll('div', attrs = {'class':'catItemImageBlock'}):
            article_l = h.find('a')
            if article_l:
                article_l = article_l['href']
                article_links.append('http://www.belrab.ru' + article_l)

            
        n += 8
    return article_links  
    



http://www.belrab.ru/rubriki/politika/item/2135-deputat-pomogaet-detyam


In [99]:
def find_info_article(article_link):
    '''function creating a file with text from an article page'''
    req = requests.get(article_link)
    soup = BeautifulSoup(req.text,'lxml')   
    title = soup.find('meta', attrs = {'name':'title'})['content']
    date = soup.find('div',  attrs = {'class':'date-vote-thor'}).text.strip()
    
    for t in soup.findAll('a', attrs = {'class':'active'}):
        if 'Рубрики' not in t['title']:
            topic = t['title']
    
    art_text = '\n\r'.join((soup.find('div',  attrs = {'class':'itemIntroText'}).text.strip(), 
                            soup.find('div',  attrs = {'class':'itemFullText'}).text.strip()))
    
    ready_text =  '\n\r'.join(('@au Noname #Too hard to extract from text', ('@ti\t' + title),
                              ('@da\t' + date), ('@topic\t' + topic),('@url\t' + article_link), ('\n\r' + art_text)))
    
    return ready_text

    
    
    
    

In [104]:
#http://www.belrab.ru/rubriki/politika?limitstart=0
def belrab(number_articles_wanted):
    '''function finding links for category pages on the site, downloading all article links from  those, downloading as much 
    articles as you state in number_articles_wanted argument'''
    num_art = 1
        
    url = 'http://www.belrab.ru/'
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'lxml')
    
    for category in soup.findAll('li', attrs = {'class':'level2'}):
        category_link = category.find('a')
        if category_link:
            for article_link in article_links(category_link['href']):
                while num_art != (number_articles_wanted + 1):
                    with open(str(num_art) + '.txt','w',encoding='utf-8') as f:
                        f.write(find_info_article(article_link))
                        num_art += 1
            
            


belrab(1000)
            


KeyboardInterrupt: 